In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

/Users/kuoteng/.pyenv/versions/3.6.4/envs/NCKU-DSAI/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
DATA_SIZE = 60000
TRAIN_SIZE = 45000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + DIGITS + 1
chars = '0123456789+-* '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
EPOCH_SIZE = 2
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

In [6]:
ctable.indices_char

{0: ' ',
 1: '*',
 2: '+',
 3: '-',
 4: '0',
 5: '1',
 6: '2',
 7: '3',
 8: '4',
 9: '5',
 10: '6',
 11: '7',
 12: '8',
 13: '9'}

# Data Generation

In [7]:
%time
questions = []
expected = []
seen = set()
print('Generating data...')

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 7.87 µs
Generating data...


In [8]:
%time
while len(questions) < DATA_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    if len(questions) % 3 == 1:
        q = '{}-{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        ans = str(a - b)
    elif len(questions) % 3 == 2:
        q = '{}+{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        ans = str(a + b)
    else:
        q = '{}*{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        ans = str(a * b)
    ans += ' ' * (MAXLEN - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.91 µs
Total addition questions: 60000


In [9]:
print(questions[:5], expected[:5])

['538*11 ', '5-202  ', '430+30 ', '548*0  ', '5-9    '] ['5918   ', '-197   ', '460    ', '0      ', '-4     ']


# Processing

In [10]:
print('Vectorization... (to the one-hot encoding)')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), MAXLEN, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, MAXLEN)

Vectorization... (to the one-hot encoding)


In [11]:
indices = np.arange(len(y))
np.random.shuffle(indices)
print(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:TRAIN_SIZE]
train_y = y[:TRAIN_SIZE]
test_x = x[TRAIN_SIZE:]
test_y = y[TRAIN_SIZE:]

print('Training Data:')
print(train_x.shape)
print(train_y.shape)

split_at = len(train_x) - len(train_x) // 10
print('split_at', split_at)
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)


[11994 29463 20284 ... 35169  6204 37574]
Training Data:
(45000, 7, 14)
(45000, 7, 14)
split_at 40500
Training Data:
(40500, 7, 14)
(40500, 7, 14)
Validation Data:
(4500, 7, 14)
(4500, 7, 14)
Testing Data:
(15000, 7, 14)
(15000, 7, 14)


In [12]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False False False False False False  True
   False False False]
  [False False False False False False  True False False False False
   False False False]
  [False False False False False False False False False False False
   False  True False]
  [False  True False False False False False False False False False
   False False False]
  [False False False False False False False False False  True False
   False False False]
  [False False False False False False False False False False False
   False  True False]
  [False False False False  True False False False False False False
   False False False]]

 [[False False False False False False False False False False False
   False False  True]
  [False False False False False False False False False  True False
   False False False]
  [False False False False  True False False False False False False
   False False False]
  [False  True False False False False False False False False False
   False Fa

# Build Model

In [13]:
print('Build model...')
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(MAXLEN))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               73216     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 7, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 7, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 7, 14)             1806      
_________________________________________________________________
activation_1 (Activation)    (None, 7, 14)             0         
Total params: 206,606
Trainable params: 206,606
Non-trainable params: 0
_________________________________________________________________


# Training

In [14]:
% time
for loop in range(50):
    print()
    print('-' * 50)
    print('Train Loop Num:', loop)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=EPOCH_SIZE,
              validation_data=(x_val, y_val),
              shuffle=True)
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.11 µs

--------------------------------------------------
Train Loop Num: 0
Train on 40500 samples, validate on 4500 samples
Epoch 1/2
40500/40500 [==============================] - 14s 351us/step - loss: 1.4266 - acc: 0.5125 - val_loss: 1.2606 - val_acc: 0.5458
Epoch 2/2
40500/40500 [==============================] - 12s 306us/step - loss: 1.2289 - acc: 0.5519 - val_loss: 1.2113 - val_acc: 0.5550
Q 868-500 T 368     ☒ -55    
Q 594-73  T 521     ☒ 189    
Q 489-47  T 442     ☒ 188    
Q 343*721 T 247303  ☒ 11198  
Q 196-85  T 111     ☒ -59    
Q 39-880  T -841    ☒ -289   
Q 138-720 T -582    ☒ -11    
Q 835*355 T 296425  ☒ 155550 
Q 363+70  T 433     ☒ 109    
Q 43-255  T -212    ☒ -155   

--------------------------------------------------
Train Loop Num: 1
Train on 40500 samples, validate on 4500 samples
Epoch 1/2
40500/40500 [==============================] - 15s 359us/step - loss: 1.1964 - acc: 0.5601 - val_loss: 1.185

40500/40500 [==============================] - 11s 272us/step - loss: 0.6962 - acc: 0.7216 - val_loss: 0.7228 - val_acc: 0.7064
Q 163+157 T 320     ☒ 368    
Q 94+49   T 143     ☒ 131    
Q 443-64  T 379     ☒ 301    
Q 961-993 T -32     ☒ -5     
Q 16*528  T 8448    ☒ 8788   
Q 912+42  T 954     ☒ 950    
Q 649+861 T 1510    ☒ 1420   
Q 544-420 T 124     ☒ 126    
Q 590-59  T 531     ☒ 557    
Q 23*47   T 1081    ☒ 101    

--------------------------------------------------
Train Loop Num: 11
Train on 40500 samples, validate on 4500 samples
Epoch 1/2
40500/40500 [==============================] - 11s 274us/step - loss: 0.6849 - acc: 0.7257 - val_loss: 0.7013 - val_acc: 0.7163
Epoch 2/2
40500/40500 [==============================] - 11s 277us/step - loss: 0.6774 - acc: 0.7291 - val_loss: 0.6991 - val_acc: 0.7145
Q 26+950  T 976     ☒ 988    
Q 807+510 T 1317    ☒ 1347   
Q 67-98   T -31     ☒ -25    
Q 76-560  T -484    ☒ -472   
Q 888+452 T 1340    ☑ 1340   
Q 9-310   T -301    ☒ -205

40500/40500 [==============================] - 15s 370us/step - loss: 0.5008 - acc: 0.8067 - val_loss: 0.5598 - val_acc: 0.7743
Epoch 2/2
40500/40500 [==============================] - 15s 362us/step - loss: 0.4923 - acc: 0.8108 - val_loss: 0.5545 - val_acc: 0.7769
Q 401*4   T 1604    ☒ 1624   
Q 975+347 T 1322    ☒ 1332   
Q 30*472  T 14160   ☒ 14340  
Q 648*79  T 51192   ☒ 51726  
Q 127+286 T 413     ☒ 499    
Q 85*560  T 47600   ☒ 47500  
Q 612*687 T 420444  ☒ 434008 
Q 88*475  T 41800   ☒ 41780  
Q 91-11   T 80      ☑ 80     
Q 119+569 T 688     ☒ 666    

--------------------------------------------------
Train Loop Num: 22
Train on 40500 samples, validate on 4500 samples
Epoch 1/2
40500/40500 [==============================] - 13s 326us/step - loss: 0.4802 - acc: 0.8166 - val_loss: 0.5627 - val_acc: 0.7761
Epoch 2/2
40500/40500 [==============================] - 14s 343us/step - loss: 0.4707 - acc: 0.8203 - val_loss: 0.5346 - val_acc: 0.7886
Q 823-553 T 270     ☒ 160    
Q 69-369

40500/40500 [==============================] - 12s 299us/step - loss: 0.3193 - acc: 0.8856 - val_loss: 0.4285 - val_acc: 0.8362
Epoch 2/2
40500/40500 [==============================] - 12s 309us/step - loss: 0.3171 - acc: 0.8861 - val_loss: 0.4265 - val_acc: 0.8386
Q 504-783 T -279    ☒ -281   
Q 682-422 T 260     ☑ 260    
Q 129*447 T 57663   ☒ 54333  
Q 80-602  T -522    ☑ -522   
Q 20*824  T 16480   ☒ 16240  
Q 63-647  T -584    ☑ -584   
Q 287*47  T 13489   ☒ 13259  
Q 124*35  T 4340    ☒ 4440   
Q 785*6   T 4710    ☒ 4790   
Q 53-296  T -243    ☑ -243   

--------------------------------------------------
Train Loop Num: 33
Train on 40500 samples, validate on 4500 samples
Epoch 1/2
40500/40500 [==============================] - 12s 301us/step - loss: 0.3107 - acc: 0.8885 - val_loss: 0.4123 - val_acc: 0.8430
Epoch 2/2
40500/40500 [==============================] - 15s 360us/step - loss: 0.3139 - acc: 0.8866 - val_loss: 0.4137 - val_acc: 0.8418
Q 52*718  T 37336   ☒ 37156  
Q 263*25

40500/40500 [==============================] - 13s 322us/step - loss: 0.2449 - acc: 0.9132 - val_loss: 0.4035 - val_acc: 0.8523
Epoch 2/2
40500/40500 [==============================] - 13s 323us/step - loss: 0.2508 - acc: 0.9112 - val_loss: 0.3886 - val_acc: 0.8577
Q 70+958  T 1028    ☑ 1028   
Q 639+87  T 726     ☑ 726    
Q 84+857  T 941     ☑ 941    
Q 6+55    T 61      ☑ 61     
Q 747*895 T 668565  ☒ 666115 
Q 85-77   T 8       ☒ 1      
Q 550*38  T 20900   ☒ 20500  
Q 83*246  T 20418   ☒ 20178  
Q 314*62  T 19468   ☒ 19088  
Q 261*340 T 88740   ☒ 81480  

--------------------------------------------------
Train Loop Num: 44
Train on 40500 samples, validate on 4500 samples
Epoch 1/2
40500/40500 [==============================] - 14s 336us/step - loss: 0.2390 - acc: 0.9163 - val_loss: 0.4209 - val_acc: 0.8517
Epoch 2/2
40500/40500 [==============================] - 14s 337us/step - loss: 0.2355 - acc: 0.9180 - val_loss: 0.3882 - val_acc: 0.8603
Q 827+72  T 899     ☒ 809    
Q 297*3 

# Testing

In [15]:
print("MSG : Prediction")
print("-" * 50)
right = 0
preds = model.predict_classes(test_x, verbose=0)
for i in range(len(preds)):
    q = ctable.decode(test_x[i])
    correct = ctable.decode(test_y[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    print('Q', q[::-1] if REVERSE else q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print(colors.ok + '☑' + colors.close, end=' ')
        right += 1
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)
print("MSG : Accuracy is {}".format(right / len(preds)))

MSG : Prediction
--------------------------------------------------
Q 91+300  T 391     ☑ 391    
Q 475*708 T 336300  ☒ 340400 
Q 232+209 T 441     ☒ 431    
Q 5-925   T -920    ☑ -920   
Q 973-76  T 897     ☒ 997    
Q 560+62  T 622     ☑ 622    
Q 20-143  T -123    ☑ -123   
Q 416-184 T 232     ☒ 238    
Q 52+611  T 663     ☑ 663    
Q 7+74    T 81      ☑ 81     
Q 630-93  T 537     ☑ 537    
Q 716+4   T 720     ☑ 720    
Q 559*526 T 294034  ☒ 298814 
Q 479*17  T 8143    ☒ 8453   
Q 53+929  T 982     ☑ 982    
Q 308-953 T -645    ☑ -645   
Q 354*37  T 13098   ☒ 13498  
Q 298*85  T 25330   ☒ 25270  
Q 995+720 T 1715    ☑ 1715   
Q 848+516 T 1364    ☑ 1364   
Q 438+97  T 535     ☑ 535    
Q 78+23   T 101     ☑ 101    
Q 299-656 T -357    ☑ -357   
Q 712+215 T 927     ☑ 927    
Q 439+450 T 889     ☒ 899    
Q 315+542 T 857     ☑ 857    
Q 524+95  T 619     ☑ 619    
Q 112-283 T -171    ☒ -161   
Q 845*35  T 29575   ☒ 29275  
Q 9-374   T -365    ☒ -369   
Q 83*464  T 38512   ☒ 38432  
Q 

Q 667+139 T 806     ☑ 806    
Q 494-75  T 419     ☑ 419    
Q 847-70  T 777     ☑ 777    
Q 931+260 T 1191    ☑ 1191   
Q 979+69  T 1048    ☑ 1048   
Q 667*699 T 466233  ☒ 478173 
Q 78*612  T 47736   ☒ 47756  
Q 576+637 T 1213    ☑ 1213   
Q 22-718  T -696    ☒ -796   
Q 4*515   T 2060    ☒ 2020   
Q 19-421  T -402    ☑ -402   
Q 313+40  T 353     ☑ 353    
Q 139+72  T 211     ☑ 211    
Q 967*655 T 633385  ☒ 638665 
Q 305-1   T 304     ☒ 204    
Q 68+817  T 885     ☑ 885    
Q 959-0   T 959     ☒ 953    
Q 25*439  T 10975   ☒ 10925  
Q 918-3   T 915     ☑ 915    
Q 22*48   T 1056    ☒ 1016   
Q 18-323  T -305    ☒ -395   
Q 230-494 T -264    ☑ -264   
Q 6-810   T -804    ☑ -804   
Q 3-43    T -40     ☒ -31    
Q 25*10   T 250     ☒ 25     
Q 230+806 T 1036    ☑ 1036   
Q 431-8   T 423     ☑ 423    
Q 253+70  T 323     ☒ 313    
Q 732-0   T 732     ☑ 732    
Q 2+609   T 611     ☑ 611    
Q 51+833  T 884     ☑ 884    
Q 10-911  T -901    ☑ -901   
Q 798*26  T 20748   ☒ 20168  
Q 317*6   

Q 184-80  T 104     ☒ 12     
Q 214+5   T 219     ☒ 229    
Q 77*499  T 38423   ☒ 38823  
Q 284+143 T 427     ☒ 527    
Q 878+56  T 934     ☒ 944    
Q 971+90  T 1061    ☑ 1061   
Q 90-641  T -551    ☒ -541   
Q 2*426   T 852     ☒ 752    
Q 1-821   T -820    ☑ -820   
Q 56*256  T 14336   ☒ 14952  
Q 44*573  T 25212   ☒ 25112  
Q 42+386  T 428     ☑ 428    
Q 473+773 T 1246    ☒ 1242   
Q 1-155   T -154    ☑ -154   
Q 223*55  T 12265   ☒ 12915  
Q 656+925 T 1581    ☑ 1581   
Q 801+297 T 1098    ☑ 1098   
Q 464-16  T 448     ☑ 448    
Q 346*957 T 331122  ☒ 336892 
Q 46*2    T 92      ☒ 96     
Q 9-497   T -488    ☑ -488   
Q 51+100  T 151     ☒ 161    
Q 38-864  T -826    ☑ -826   
Q 465+9   T 474     ☑ 474    
Q 838*37  T 31006   ☒ 39646  
Q 132-29  T 103     ☑ 103    
Q 194-53  T 141     ☒ 143    
Q 548*133 T 72884   ☒ 77714  
Q 743+3   T 746     ☑ 746    
Q 95+425  T 520     ☒ 510    
Q 566+243 T 809     ☒ 819    
Q 86+271  T 357     ☒ 367    
Q 29+355  T 384     ☑ 384    
Q 424*882 

Q 649+579 T 1228    ☑ 1228   
Q 816+37  T 853     ☑ 853    
Q 884+40  T 924     ☑ 924    
Q 917*117 T 107289  ☒ 108699 
Q 241-28  T 213     ☑ 213    
Q 539+25  T 564     ☑ 564    
Q 373*220 T 82060   ☒ 72140  
Q 85*1    T 85      ☒ 75     
Q 572-939 T -367    ☒ -361   
Q 55+79   T 134     ☑ 134    
Q 131-9   T 122     ☒ 132    
Q 494+953 T 1447    ☑ 1447   
Q 184-466 T -282    ☒ -288   
Q 54*43   T 2322    ☒ 2332   
Q 15-74   T -59     ☑ -59    
Q 12*796  T 9552    ☒ 9592   
Q 539*45  T 24255   ☒ 23195  
Q 279*930 T 259470  ☒ 263306 
Q 672*419 T 281568  ☒ 273188 
Q 34+342  T 376     ☑ 376    
Q 630*911 T 573930  ☒ 583790 
Q 57*893  T 50901   ☒ 51801  
Q 968*85  T 82280   ☒ 83460  
Q 336+54  T 390     ☒ 380    
Q 1*310   T 310     ☑ 310    
Q 335*36  T 12060   ☒ 12360  
Q 769+25  T 794     ☒ 894    
Q 162-60  T 102     ☑ 102    
Q 2*903   T 1806    ☒ 1836   
Q 450*62  T 27900   ☒ 28800  
Q 940+812 T 1752    ☒ 1842   
Q 485-516 T -31     ☑ -31    
Q 105+21  T 126     ☑ 126    
Q 78-376  

Q 0+643   T 643     ☑ 643    
Q 202-754 T -552    ☑ -552   
Q 14*54   T 756     ☒ 716    
Q 838*489 T 409782  ☒ 496482 
Q 491+887 T 1378    ☑ 1378   
Q 806+24  T 830     ☒ 820    
Q 1-396   T -395    ☑ -395   
Q 737+18  T 755     ☑ 755    
Q 758-482 T 276     ☑ 276    
Q 712-868 T -156    ☒ -146   
Q 571*768 T 438528  ☒ 440138 
Q 2*420   T 840     ☒ 820    
Q 445-418 T 27      ☒ 31     
Q 605*7   T 4235    ☒ 4165   
Q 69*418  T 28842   ☒ 28762  
Q 39+126  T 165     ☑ 165    
Q 3*896   T 2688    ☒ 2648   
Q 58-291  T -233    ☑ -233   
Q 214+76  T 290     ☒ 280    
Q 33+957  T 990     ☒ 980    
Q 41*740  T 30340   ☒ 30940  
Q 781*5   T 3905    ☒ 3855   
Q 2*36    T 72      ☒ 74     
Q 284+46  T 330     ☑ 330    
Q 78-30   T 48      ☑ 48     
Q 195-361 T -166    ☒ -106   
Q 61+219  T 280     ☒ 270    
Q 723-560 T 163     ☒ 179    
Q 23-683  T -660    ☒ -650   
Q 519-936 T -417    ☒ -429   
Q 912*776 T 707712  ☒ 793332 
Q 96-381  T -285    ☑ -285   
Q 751+94  T 845     ☒ 835    
Q 32*268  

...

Q 49+154  T 203     ☑ 203    
Q 528*73  T 38544   ☒ 38424  
Q 31+791  T 822     ☑ 822    
Q 669-2   T 667     ☑ 667    
Q 635*70  T 44450   ☒ 44650  
Q 161+893 T 1054    ☑ 1054   
Q 510-86  T 424     ☒ 434    
Q 94-876  T -782    ☑ -782   
Q 43-94   T -51     ☑ -51    
Q 70*166  T 11620   ☒ 11280  
Q 2-265   T -263    ☑ -263   
Q 6-477   T -471    ☑ -471   
Q 875+393 T 1268    ☑ 1268   
Q 2-329   T -327    ☒ -323   
Q 616-34  T 582     ☑ 582    
Q 46*178  T 8188    ☒ 8688   
Q 949+963 T 1912    ☑ 1912   
Q 284*37  T 10508   ☒ 10488  
Q 6-478   T -472    ☑ -472   
Q 701*37  T 25937   ☒ 25107  
Q 964+84  T 1048    ☑ 1048   
Q 169+599 T 768     ☒ 788    
Q 987+34  T 1021    ☑ 1021   
Q 587-488 T 99      ☒ 109    
Q 647-48  T 599     ☑ 599    
Q 362-28  T 334     ☑ 334    
Q 823*99  T 81477   ☒ 80377  
Q 859*633 T 543747  ☒ 538967 
Q 1*225   T 225     ☑ 225    
Q 798*43  T 34314   ☒ 33364  
Q 570*56  T 31920   ☒ 31280  
Q 302+1   T 303     ☒ 203    
Q 53+669  T 722     ☑ 722    
Q 993*959 

Q 308-87  T 221     ☒ 211    
Q 472*959 T 452648  ☒ 450808 
Q 35+961  T 996     ☒ 906    
Q 38-56   T -18     ☑ -18    
Q 33*82   T 2706    ☒ 2716   
Q 131-16  T 115     ☒ 105    
Q 53-190  T -137    ☒ -127   
Q 12+960  T 972     ☒ 976    
Q 67-788  T -721    ☒ -729   
Q 639*920 T 587880  ☒ 587540 
Q 630-62  T 568     ☑ 568    
Q 100*104 T 10400   ☒ 11600  
Q 6+8     T 14      ☑ 14     
Q 302+218 T 520     ☒ 510    
Q 46+162  T 208     ☑ 208    
Q 341*68  T 23188   ☒ 22308  
Q 27-397  T -370    ☑ -370   
Q 34*134  T 4556    ☒ 4456   
Q 878*527 T 462706  ☒ 450146 
Q 33+778  T 811     ☑ 811    
Q 63+967  T 1030    ☑ 1030   
Q 13+737  T 750     ☒ 740    
Q 301-881 T -580    ☑ -580   
Q 884-43  T 841     ☒ 831    
Q 61*207  T 12627   ☒ 12667  
Q 558+91  T 649     ☑ 649    
Q 67+218  T 285     ☑ 285    
Q 48-48   T 0       ☑ 0      
Q 3+645   T 648     ☒ 658    
Q 241+925 T 1166    ☑ 1166   
Q 554*85  T 47090   ☒ 48510  
Q 377*23  T 8671    ☒ 8751   
Q 62+948  T 1010    ☑ 1010   
Q 694-71  

Q 71+752  T 823     ☑ 823    
Q 50-77   T -27     ☑ -27    
Q 167*737 T 123079  ☒ 122859 
Q 290*238 T 69020   ☒ 74860  
Q 1+776   T 777     ☑ 777    
Q 4+887   T 891     ☑ 891    
Q 90+123  T 213     ☑ 213    
Q 553*55  T 30415   ☒ 30645  
Q 988+29  T 1017    ☑ 1017   
Q 771-536 T 235     ☒ 345    
Q 289-36  T 253     ☑ 253    
Q 68-176  T -108    ☑ -108   
Q 265+909 T 1174    ☒ 1154   
Q 122-77  T 45      ☒ 55     
Q 801+134 T 935     ☒ 945    
Q 586*403 T 236158  ☒ 234618 
Q 350*3   T 1050    ☒ 1150   
Q 89*732  T 65148   ☒ 64648  
Q 667-327 T 340     ☒ 330    
Q 603-794 T -191    ☒ -171   
Q 10*582  T 5820    ☒ 5940   
Q 96*61   T 5856    ☑ 5856   
Q 612+987 T 1599    ☒ 1693   
Q 511+442 T 953     ☑ 953    
Q 466-749 T -283    ☒ -273   
Q 213*16  T 3408    ☒ 3968   
Q 910+946 T 1856    ☑ 1856   
Q 5*894   T 4470    ☒ 4490   
Q 638-755 T -117    ☒ -119   
Q 676-217 T 459     ☒ 469    
Q 82-46   T 36      ☑ 36     
Q 644*620 T 399280  ☒ 497540 
Q 14+90   T 104     ☑ 104    
Q 146+108 

Q 567-13  T 554     ☒ 644    
Q 470*529 T 248630  ☒ 243490 
Q 37-692  T -655    ☑ -655   
Q 32*831  T 26592   ☒ 26452  
Q 45+272  T 317     ☑ 317    
Q 42-82   T -40     ☑ -40    
Q 302-42  T 260     ☒ 250    
Q 47-317  T -270    ☒ -260   
Q 93-713  T -620    ☑ -620   
Q 246+304 T 550     ☑ 550    
Q 12-998  T -986    ☒ 9996   
Q 822*4   T 3288    ☒ 3248   
Q 0-637   T -637    ☒ -647   
Q 1*49    T 49      ☒ 39     
Q 66+51   T 117     ☑ 117    
Q 532*703 T 373996  ☒ 373102 
Q 91-92   T -1      ☒ -4     
Q 747+94  T 841     ☒ 831    
Q 52+987  T 1039    ☑ 1039   
Q 715+637 T 1352    ☑ 1352   
Q 884*25  T 22100   ☒ 22260  
Q 742*4   T 2968    ☒ 2908   
Q 787-99  T 688     ☑ 688    
Q 88-972  T -884    ☒ -894   
Q 400*9   T 3600    ☒ 3400   
Q 4-798   T -794    ☑ -794   
Q 98*936  T 91728   ☒ 91358  
Q 902-297 T 605     ☒ 627    
Q 5+354   T 359     ☑ 359    
Q 723*936 T 676728  ☒ 660488 
Q 556+5   T 561     ☑ 561    
Q 728*63  T 45864   ☒ 45684  
Q 630+813 T 1443    ☑ 1443   
Q 23+575  

Q 966*735 T 710010  ☒ 711110 
Q 87+578  T 665     ☑ 665    
Q 341-180 T 161     ☑ 161    
Q 910-0   T 910     ☒ 900    
Q 347-21  T 326     ☑ 326    
Q 59-944  T -885    ☒ -879   
Q 132-48  T 84      ☒ 82     
Q 676*35  T 23660   ☒ 23110  
Q 294+549 T 843     ☑ 843    
Q 425+890 T 1315    ☑ 1315   
Q 3-874   T -871    ☑ -871   
Q 10-430  T -420    ☒ -430   
Q 55-775  T -720    ☑ -720   
Q 22+738  T 760     ☑ 760    
Q 6-511   T -505    ☑ -505   
Q 378+21  T 399     ☒ 499    
Q 56-542  T -486    ☑ -486   
Q 53+174  T 227     ☒ 221    
Q 85*808  T 68680   ☒ 68440  
Q 792-286 T 506     ☑ 506    
Q 69*602  T 41538   ☒ 41758  
Q 453*69  T 31257   ☒ 31667  
Q 38-866  T -828    ☑ -828   
Q 560-93  T 467     ☒ 477    
Q 206+7   T 213     ☑ 213    
Q 91+907  T 998     ☒ 990    
Q 844+205 T 1049    ☑ 1049   
Q 313-70  T 243     ☑ 243    
Q 78*805  T 62790   ☒ 62430  
Q 813*3   T 2439    ☒ 2499   
Q 488*8   T 3904    ☒ 4044   
Q 639-593 T 46      ☒ 44     
Q 900*686 T 617400  ☒ 604800 
Q 917-91  

In [16]:
# print("MSG : Prediction")
# test_x = ["555+275", "860+7  ", "340+29 "]
# test_y = ["830 ", "867 ", "369 "]
# x = np.zeros((len(test_x), MAXLEN, len(chars)), dtype=np.bool)
# y = np.zeros((len(test_y), DIGITS + 1, len(chars)), dtype=np.bool)
# for j, (i, c) in enumerate(zip(test_x, test_y)):
#     x[j] = ctable.encode(i, MAXLEN)
#     y[j] = ctable.encode(c, DIGITS + 1)